# Расчёт времени нахождения в терапевтическом диапазоне методом усреднения по всем значениям МНО

**Описание проекта:** <br>
    Пациенты, перенесшие операции на сердце принимают кроверазжижающие препараты на основе варфарина. Варфарин очень опасное вещество, так как его передозировка может вызывать кровотечения как наружние, так и внутренние. Если же пациент принимает слишклм малую дозу то возникает риск образования тромбов. Параметром, который позволяет контролированть правильность дозировки, является МНО (международное нормализованное отношение). У каждого пациента, в зависимости от его диагноза, должен быть свой терапевтический диапазон в котором должен находится этот параметр, плюс врач может его скорректировать по конкретного пациента, исходя из его истории лечения. У разных людей встречается разная чувствительность к препаратам, содержащим варфарин, поэтому дозировка для каждого пациента подбирается индивидуально на основе его предыдущих дозировок и значений МНО. МНО сдаётся один раз в 10-20 дней и может как попадать в целевой диапазон, так и выпадать из него. Критерием эффективности назначаемой дозировки является такой параметр, как TTR (Time in Therapeutic Range). Проблема заключается в том, что нет утвержденных стандартов его расчёта.
    
**Цель:**
   1. Изучить данные и составить алгоритм расчёта TTR.
   2. Учесть, что у пациента может быть несколько заболеваний, и несколько соответствующих им целевых диапазонов.
   3. Иметь в виду, что врач может внести изменения в терапевтический диапазон.
   4. Рассчитать TTR как отношение МНО, вошедших в индивидуалльный целевой диапазон к общему количеству взятых МНО.
   5. Сгруппировать результаты по годам отдельно для пациентов, сдающих анализ дома, и для пациентов с собственным прибором.
   6. Вывести график
   
**Этапы исследования:**

   1. Загрузка данных и изучение общей информации,
   2. Анализ данных,
   3. Добавление новых признаков,
   4. Расчёт
   5. Презентация результатов

**Импортируем библиотеки:**

In [18]:
import pandas as pd 
import plotly.express as px

**Загружаем данные:**

In [19]:
inr = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_inr.csv')
indications = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_indications.csv')
treatment_data = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_treatment_data.csv')
patient_institutions = pd.read_csv('C:/Users/volsi/Documents/Filezilla/dwh_dump/dwh_ts_kazan_patient_institutions.csv')

**Посмотрим, из чего состоит таблица inr:**

In [20]:
inr.head() # For TTR calculation

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...
2,6026394360,6059271230,2024-04-12 00:00:00,3.1,LAB,CSP,2.0,6140543514,2024-04-18 08:21:25.910798,return ''.sprintf(gettext('Huidige INR %s ligt...
3,6026394638,6059368614,2023-11-24 11:42:00,3.0,ZMA,CSP,2.0,6137177792,2023-11-24 11:42:44.286527,return ''.gettext('Er kon geen doseervoorstel ...
4,6026138194,6026318204,2018-03-23 00:00:00,4.4,LAB,CSP,3.0,6026326660,2018-03-23 11:00:54.859975,return ''.gettext('Er zijn minder dan 2 opeenv...


**Здесь все значения МНО с датой, временем и типом терапевтического диапазона, назначаемого автоматически, но нет информации об индивидуальном терапевтическом диапазоне.**

**Посмотрим, из чего состоит таблица indications:**

In [21]:
indications.head() # automatic therapeutic range can be taken from here

,peso_id,trbh_id,effective_from,effective_to,code,name,remark,type,location,target_range,id
0,6026136398,6026185844,2016-11-28 00:00:00,NaN,134,Onstabiele angina pectoris,NaN,NaN,NaN,2,6026185868
1,6026137016,6026218258,2017-03-10 00:00:00,NaN,1049,Mechanische mitralis hartklepprothese + Risico...,NaN,мех,мк,2,6057488560
2,6026137406,6026250702,2017-08-15 00:00:00,NaN,1021,Bioklepprothese + Risicofactor,NaN,NaN,NaN,1,6055527868
3,6026137934,6026292136,2022-10-23 00:00:00,NaN,1029,Overige (zeldzame) indicaties (streefgebied 1),Протезирование ВОА,NaN,NaN,1,6123427048
4,6026138020,6026301532,2018-01-22 00:00:00,NaN,1040,AF + (cerebrale) embolie,NaN,NaN,NaN,1,6035999650


**Здесь указан автоматический терапевтический диапазон для каждого пациента. Важно, что у него есть время действия. Для одного пациента может существовать несколько строк.**

**Можем сразу найти максимальный целевой диапазон у пациента, так как учитывается именно он:**

In [22]:
indications.groupby('peso_id')['target_range'].max() # maximum automatic therapeutic range

peso_id
6026136398    2
6026136402    1
6026136420    2
6026136424    2
6026136428    2
             ..
6028057454    2
6028057456    2
6028057458    1
6028057472    1
6028057474    1
Name: target_range, Length: 9272, dtype: int64

**Посмотрим, из чего состоит таблица treatment_data:**

In [23]:
treatment_data.info() # personal target range is here

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9463 entries, 0 to 9462
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   peso_id                        9463 non-null   int64  
 1   trbh_id                        9463 non-null   int64  
 2   effective_from                 9463 non-null   object 
 3   effective_to                   2683 non-null   object 
 4   duration_days                  90 non-null     float64
 5   duration_auto_stop             9463 non-null   object 
 6   status                         9463 non-null   object 
 7   dosing_max_period              9463 non-null   int64  
 8   dosing_margin_days             9463 non-null   int64  
 9   target_range_deviation_lower   1434 non-null   float64
 10  target_range_deviation_upper   1434 non-null   float64
 11  target_range_deviation_reason  1434 non-null   object 
 12  curr_status_selfmeasurement    9463 non-null   o

**В этом файле появляется информация об изменениях целевого диапазона МНО, внесенного врачом**

**Отфильтруем пациентов с измененным целевым диапазоном:**

In [24]:
treatment_data.query('target_range_deviation_lower.notna()') # the list of patients with not empty personal target range

,peso_id,trbh_id,effective_from,effective_to,duration_days,duration_auto_stop,status,dosing_max_period,dosing_margin_days,target_range_deviation_lower,...,curr_status_selfdosing,curr_status_regular,curr_status_training,curr_status_home_poli,curr_status_dosing_dr,curr_status_new_patient,permanent_message,puncture_list_message,call_list_message,reason
6,6026393996,6059129490,2020-04-27 00:00:00,2021-09-03 00:00:00,NaN,N,COMPLETED,30,10,2.5,...,N,N,N,N,NaN,NaN,"пластика ТК, стентирование левой ПА в 2014 году",NaN,NaN,return ''.'дубль';
21,6026351208,6049591760,2019-11-18 00:00:00,2021-09-03 00:00:00,NaN,N,COMPLETED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'дубль';
23,6026138730,6026365748,2018-06-21 00:00:00,2018-08-13 00:00:00,NaN,N,DIED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'Просто умер';
40,6026136880,6026209870,2017-02-03 00:00:00,2019-09-07 00:00:00,NaN,N,COMPLETED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return gettext('Foutief ingevoerd').'';
42,6026299230,6043411018,2019-10-06 00:00:00,2023-05-13 00:00:00,184.0,J,COMPLETED,30,10,2.0,...,N,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'кардиолог отменил варфарин';
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9410,6026137296,6026243432,2017-07-19 00:00:00,NaN,NaN,N,ACTIVE,30,10,2.0,...,N,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9442,6027962856,6116736742,2022-12-12 00:00:00,NaN,NaN,N,ACTIVE,30,10,1.5,...,N,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9446,6026137302,6026243758,2017-07-20 00:00:00,NaN,NaN,N,ACTIVE,30,10,2.5,...,N,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
9455,6027926750,6103653352,2022-09-12 00:00:00,NaN,NaN,N,ACTIVE,30,10,2.5,...,N,N,NaN,N,NaN,N,"кардиолог МКДЦ (целевой уровень МНО 2,5-3,5)",NaN,NaN,NaN


**Объеденим информацию об автоматическом и ручном целевых диапазонах:**

In [25]:
ranges = treatment_data.merge(indications.groupby('peso_id')['target_range'].max(),on = 'peso_id') # here we have as personal as automatic target range
ranges

,peso_id,trbh_id,effective_from,effective_to,duration_days,duration_auto_stop,status,dosing_max_period,dosing_margin_days,target_range_deviation_lower,...,curr_status_regular,curr_status_training,curr_status_home_poli,curr_status_dosing_dr,curr_status_new_patient,permanent_message,puncture_list_message,call_list_message,reason,target_range
0,6026136398,6026185844,2016-11-28 00:00:00,2016-11-28 00:00:00,NaN,N,DIED,21,3,NaN,...,N,N,N,NaN,Y,Пациент просто душка,NaN,NaN,return ''.'Он был просто душкой';,2
1,6026136402,6026186056,2016-11-28 00:00:00,2016-11-30 00:00:00,NaN,N,DIED,21,3,NaN,...,N,N,N,N,Y,Это черт во плоти,NaN,NaN,return ''.'тестовый';,1
2,6026136790,6026205794,2017-01-23 00:00:00,2021-09-16 00:00:00,NaN,N,COMPLETED,30,10,NaN,...,N,N,N,NaN,NaN,NaN,NaN,NaN,return ''.'не явка ';,2
3,6026136450,6026188328,2016-12-01 00:00:00,2016-12-02 00:00:00,NaN,N,DIED,21,3,NaN,...,N,N,N,NaN,Y,NaN,NaN,NaN,return ''.'бог дал бог взял';,2
4,6026136420,6026187004,2016-11-30 00:00:00,2016-12-02 00:00:00,NaN,N,COMPLETED,21,3,NaN,...,N,N,N,NaN,Y,тестовый пациент,NaN,NaN,return ''.'212313';,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455,6027855788,6082825142,2021-12-01 00:00:00,NaN,NaN,N,ACTIVE,30,10,NaN,...,Y,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,1
9456,6026433462,6069098896,2021-07-16 00:00:00,NaN,NaN,N,SUSPENDED,30,10,2.5,...,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1
9457,6026261980,6039449544,2019-08-28 00:00:00,NaN,NaN,N,ACTIVE,30,10,NaN,...,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2
9458,6028009272,6137163088,2023-11-23 00:00:00,NaN,NaN,N,ACTIVE,30,10,NaN,...,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1


**Добавим новые столбцы и заполним их нулями для удобства:**

In [26]:
ranges['final_lower']= [0 for i in range (len(ranges))]
ranges['upper_lower']= [0 for i in range (len(ranges))]

**Посчитаем итоговый целевой диапазон для каждого пациента. Будем считать введенные врачом данные приоритетными:**

In [27]:
for i in range(len(ranges)): # here the final taget range is calculated
    if ranges.loc[i,'target_range_deviation_lower']>0:
        ranges.loc[i,'final_lower'] = ranges.loc[i,'target_range_deviation_lower']
        ranges.loc[i,'final_upper'] = ranges.loc[i,'target_range_deviation_upper']
    elif ranges.loc[i,'target_range'] == 2:
        ranges.loc[i,'final_lower'] = 2.5
        ranges.loc[i,'final_upper'] = 3.5
    elif ranges.loc[i,'target_range'] == 1:
        ranges.loc[i,'final_lower'] = 2.0
        ranges.loc[i,'final_upper'] = 3.0
final_ranges = ranges[['peso_id', 'target_range_deviation_lower', 'target_range_deviation_upper', 'target_range','final_lower','final_upper']]
final_ranges

,peso_id,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper
0,6026136398,NaN,NaN,2,2.5,3.5
1,6026136402,NaN,NaN,1,2.0,3.0
2,6026136790,NaN,NaN,2,2.5,3.5
3,6026136450,NaN,NaN,2,2.5,3.5
4,6026136420,NaN,NaN,2,2.5,3.5
...,...,...,...,...,...,...
9455,6027855788,NaN,NaN,1,2.0,3.0
9456,6026433462,2.5,3.0,1,2.5,3.0
9457,6026261980,NaN,NaN,2,2.5,3.5
9458,6028009272,NaN,NaN,1,2.0,3.0


**А теперь привяжем финальный целевой диапазон к таблице МНО:**

In [28]:
final_inr = inr.merge(final_ranges, on = 'peso_id') # All INRs with final target range
#final_inr['peso_id'].nunique()
final_inr

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167152,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5
167153,6026181484,6030459068,2018-12-11 00:00:00,1.7,LAB,CSP,NaN,6031693262,NaN,NaN,2.5,3.0,1,2.5,3.0
167154,6026421830,6066926304,2021-03-24 00:00:00,2.3,LAB,HCSP,NaN,6066926332,NaN,NaN,2.0,3.0,1,2.0,3.0
167155,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5


**Добавим столбец с признаком 'входит/не входит в целевой диапазон':**

In [29]:
for i in range(len(final_inr)): # here the new column "inside" is filled in
    if final_inr.loc[i,'final_lower']<=final_inr.loc[i,'inr']<=final_inr.loc[i,'final_upper']:
        final_inr.loc[i,'inside'] = 100
    else:
        final_inr.loc[i,'inside'] = 0
final_inr        

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167152,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0
167153,6026181484,6030459068,2018-12-11 00:00:00,1.7,LAB,CSP,NaN,6031693262,NaN,NaN,2.5,3.0,1,2.5,3.0,0.0
167154,6026421830,6066926304,2021-03-24 00:00:00,2.3,LAB,HCSP,NaN,6066926332,NaN,NaN,2.0,3.0,1,2.0,3.0,100.0
167155,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0


**Добавим столбец с годом взятия МНО для последующей группировки:**

In [30]:
final_inr['year'] = final_inr['inr_date'].astype("datetime64[ns]").dt.to_period("Y") #here the new column for year of INR is added
final_inr

,peso_id,trbh_id,inr_date,inr,method_cd,ind_status,list_nr,id,dosing_process_timestamp,reason_dosing_list,target_range_deviation_lower,target_range_deviation_upper,target_range,final_lower,final_upper,inside,year
0,6027991086,6131492478,2023-06-10 09:53:00,4.1,ZMA,CSP,3.0,6131840490,2023-06-10 09:54:57.089723,return ''.gettext('Er zijn minder dan 2 opeenv...,2.5,3.5,1,2.5,3.5,0.0,2023
1,6026141152,6026512906,2023-09-01 06:49:00,1.7,ZMA,CSP,2.0,6134525496,2023-09-01 06:49:30.63231,return ''.sprintf(gettext('Huidige INR %s ligt...,2.5,3.5,1,2.5,3.5,0.0,2023
2,6026141152,6026512906,2020-04-29 00:00:00,3.2,ZMA,CSP,2.0,6059205866,2020-04-29 17:35:39.170686,return ''.sprintf(gettext('Vorige INR %s (%s) ...,2.5,3.5,1,2.5,3.5,100.0,2020
3,6026141152,6026512906,2020-05-28 00:00:00,2.5,ZMA,CSP,2.0,6059638402,2020-05-28 14:24:20.39494,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,2020
4,6026141152,6026512906,2020-07-08 00:00:00,2.6,ZMA,CSP,2.0,6060016868,2020-07-08 14:59:13.154197,return ''.sprintf(gettext('Aantal wegzenddagen...,2.5,3.5,1,2.5,3.5,100.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167152,6027983126,6127118628,2023-03-28 00:00:00,1.8,LAB,HCSP,NaN,6127118654,NaN,NaN,NaN,NaN,2,2.5,3.5,0.0,2023
167153,6026181484,6030459068,2018-12-11 00:00:00,1.7,LAB,CSP,NaN,6031693262,NaN,NaN,2.5,3.0,1,2.5,3.0,0.0,2018
167154,6026421830,6066926304,2021-03-24 00:00:00,2.3,LAB,HCSP,NaN,6066926332,NaN,NaN,2.0,3.0,1,2.0,3.0,100.0,2021
167155,6027973170,6120390772,2023-10-10 00:00:00,2.5,LAB,CSP,NaN,6136278298,NaN,NaN,NaN,NaN,2,2.5,3.5,100.0,2023


**Рассчитаем TTR у всех пациентов с разбивкой по годам:**

In [31]:
final_inr.groupby(['year','method_cd'])['inside'].mean() #TTR

year  method_cd
2014  LAB           0.000000
2015  LAB          50.000000
2016  LAB          48.979592
2017  LAB          51.554314
      ZMA          33.333333
2018  LAB          48.455975
      ZMA          33.333333
2019  LAB          50.305693
      ZMA          38.068182
2020  LAB          53.720029
      ZMA          52.220114
2021  LAB          54.310502
      ZMA          60.069009
2022  LAB          54.353473
      ZMA          62.192530
2023  LAB          56.205706
      ZMA          58.846454
2024  LAB          59.335850
      ZMA          61.892131
Name: inside, dtype: float64

In [32]:
TTR = final_inr.groupby(['year','method_cd'])['inside'].mean().reset_index()
TTR.columns = ["Год", 'Тип_измерения', 'TTR']
TTR['Год'] = TTR['Год'].dt.year
TTR

,Год,Тип_измерения,TTR
0,2014,LAB,0.000000
1,2015,LAB,50.000000
2,2016,LAB,48.979592
3,2017,LAB,51.554314
4,2017,ZMA,33.333333
5,2018,LAB,48.455975
6,2018,ZMA,33.333333
7,2019,LAB,50.305693
8,2019,ZMA,38.068182
9,2020,LAB,53.720029


In [33]:
fig = px.line(TTR, y = "TTR", x = "Год", color = "Тип_измерения", title='TTR у всех пациентов с разбивкой по годам')
fig.show()

**Создадим для наглядности сводную таблицу TTR по годам для пациентов, сдающих анализ дома, и для пациентов с собственным прибором:**

In [34]:
final_inr.pivot_table(index ='method_cd', columns = 'year', values = 'inside', aggfunc = 'mean' ) #TTR pivot table

year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
method_cd,,,,,,,,,,,
LAB,0.0,50.0,48.979592,51.554314,48.455975,50.305693,53.720029,54.310502,54.353473,56.205706,59.335850
ZMA,NaN,NaN,NaN,33.333333,33.333333,38.068182,52.220114,60.069009,62.192530,58.846454,61.892131


## Выводы:
   1. Программа расчёта TTR методом усреднения по всем значениям МНО написана. При появлении новых данных достаточно её ещё раз запустить, чтобы получить актуальную информацию.
   2. Результаты анализа данных сведены в таблицы и визулизированы при помощи графика. 
   3. Из графика и таблиц следует, что:
      - у пациентов, сдающих анализы в лаборатрии, есть данные начиная с 2014 года;
      - у пациентов, сдающих анализы в лаборатрии, есть данные начиная с 2017 года;
      - пациенты, сдающие анализы в лаборатрии, в 2017ом году имели более высокий TTR, но после 2020 года самоизмерямые пациенты обогнали первых по этому параметру;
      - независимо от способа сдачи МНО, отчётливо виден рост TTR из года в год.